# De Novo Enzyme Design with Rosetta

In the previous tutorial, we explored algorithms that manipulate the identity and degrees of freedom in individual amino acid residues to generate entire proteins with new structures and functions. We will further explore how we can alter protein sequence and struture to create proteins with exciting new funcitons. In this tutorial, we will build off existing protein structures to impart new function; in this case, we will be introducing the ability to catalyze chemistry into a protein. In other words, we will be building an enzyme from scratch!

In this tutorial, you will become familiar with:

* How enzymes catalyze chemical reactions
* Defining a theozyme and building it into a protein
* Additional protein design strategies
* Structural quality filters for protein design

This tutorial is an adaptation of the book chapter [Computational Introduction of Catalytic Activity into Proteins](https://link.springer.com/protocol/10.1007/978-1-4939-3569-7_13) by Steve J. Bertolani, Dylan Alexander Carlin, and Justin B. Siegel in the book *Computational Design of Ligand Binding Proteins*.

In [10]:
# Package and module imports
import os
import shutil

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('seaborn-white')

import pyrosetta
import pyrosetta.rosetta as rosetta
from pyrosetta import toolbox

from ipywidgets import interact, interactive, fixed, interact_manual, HBox, Label
import ipywidgets as widgets
from IPython.display import display

# PyRosetta init
LG1_params_path = os.path.join('Assets', 'Tutorial_4', 'LG1.params')
pyrosetta.init(f'''
                -ex1 -ex2 -extrachi_cutoff 0 -mute all -extra_res_fa {LG1_params_path}
                -enzdes:parser_read_cloud_pdb -run:preserve_header
                ''')
sfxn = rosetta.core.scoring.get_score_function()
pymover = rosetta.protocols.moves.PyMOLMover()

# Widget init
output = widgets.Output()

PyRosetta-4 2019 [Rosetta devel 2019.40.post.dev+40.master.aeda51ea830d1d315912d2563a754cb38d9e49b8 2019-10-07T17:21:58] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


## A Quick Review on Enzymes

Enzymes are special proteins that possess the ability to catalyze chemical reactions and transform substrates/reactants into products. We will take a cursory look at the reaction catalyzed by [carbonic anhydrase](https://pdb101.rcsb.org/motm/49), an important enzyme that helps solublize carbon dioxide in our blood by converting carbon dioxide and water into carbonic acid. To catalyze this reaction, carbonic anhydrase possesses a pocket, called the active site, that can transiently bind the substrate. 

<img src="Assets/Tutorial_4/CA_full.png" width="600" align="center"/>

The active site contains a set of amino acid residues, and sometimes a [cofactor/prothetic group](https://en.wikipedia.org/wiki/Cofactor_(biochemistry)), in a specific configuration that permits chemistry to occur (e.g. acid-base catalysis, metal-ion catalysis). Carbonic anhydrase has three histidine residues coordinating a zinc ion (large purple/grey sphere). This zinc ion coordinates a hydroxide ion (red sphere) that is primed for nucleophilic attack of carbon dioxide bound in the active site.

<br>
<img src="Assets/Tutorial_4/CA_catalyticresidues.png" width="400" align="center"/>
<br>

The progression of a reaction catalyzed by an enzyme can be visualized on a reaction coordinate, where the x-axis represents the reaction over time and the y-axis depicts the free energy of the system:

<br>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Carbonic_anhydrase_reaction_in_tissue.svg/504px-Carbonic_anhydrase_reaction_in_tissue.svg.png" width="600" align="center" alt="Source: Wikimedia"/>
<br>

The hump between reactants and products in the reaction coordinate is the **activation energy**; this is the minimum amount of energy required for the reaction to proceed forward. The peak of the reaction coordinate is where the **transistion state** is formed; the transition state is a transient high-energy strained state where bonds are being formed/broken to interconvert between substrates and products.

It is here where enzymes act: enzymes increase the rate of reaction by stabilizing the transition state and lowering the activation energy for the reaction. Note in the the reaction coordinate above that the activation energy for the enzyme-catalyzed reaction (blue path) has a significantly lower activation energy that the uncatalyzed reaction (red path). 

The rate of a reaction (from a macroscopic perspective) can be calculated using the Arrhenius Rate Law, which takes the form: 

<br><center>$k=Ae^{{-E_a}/{RT}}$</center><br>
    
where $k$ is the reaction rate, $-E_a$ is the activation energy, $R$ is gas constant, and $T$ is temperature. $A$ is a parameter called the pre-exponential factor that is emperically determined and relates temperature to the reaction rate. Using this equation, it is straightforward to see how lowering the activation energy of a reaction will lead to an increase in reaction rate.

Now that we have the basics down, let's investigate what a transition state looks like and how we can use it to engineer catalyic function into proteins from scratch.

### Additional Resources
[Here](https://www.ncbi.nlm.nih.gov/books/NBK22599/) is a reference if you want to learn more about the reaction catalyzed by carbonic anhydrase. *Modern Physical Organic Chemistry* by Eric V. Anslyn and Dennis A. Dougherty is also a great resource to learn about transition state theory and catalysis (and isn't nearly as daunting and the title makes it seem). Another outstanding resource is *Structure and Mechanism in Protein Science* by Alan Fersht.

## What is a Theozyme?

A [theozyme](http://blueline.ucdavis.edu/2ndTier/3rdTier/Theozymes.html) (**theo**retical en**zyme**) is a computationally-determined configuration of functional groups predicted to stabilize a given transition state. While theozymes were [originally proposed](https://www.sciencedirect.com/science/article/abs/pii/S1367593198801129) for elucidating the functional contributions of active site residues toward catalysis in enzymes, theozymes have been adopted for forward design of new protein function. [This review](https://www.sciencedirect.com/science/article/abs/pii/S0958166914000652) covers the success of *de novo* designed enzymes for the [diels-alder](https://science.sciencemag.org/content/329/5989/309), [retro-aldol](https://science.sciencemag.org/content/319/5868/1387), and [kemp elimination](https://www.ncbi.nlm.nih.gov/pubmed/18354394) reactions. These successses all use theozymes determined by quantum mechanical calculations to build a functional active site into an existing protein structure.

For this tutorial, we will be using a theozyme derived from PDB ID [2JIE](https://www.rcsb.org/structure/2JIE). 2JIE is a crystal structure of a beta-glucosidase from *Bacillus polymyxa* bound to the transition state inhibitor 2-deoxy-2-fluoro-alpha-D-glucopyranose. Beta-glucosidases catalyze the hydrolysis of glycosidic bonds. Since a transition state inhibitor resembles the transition state of the reaction that this enzyme catalyzes, we can infer the contact geometries of the catalytic residues with the transition state for a new substrate, in this case 4-para-nitrophenyl-beta-D-glucopyranoside (pNPG), by observing the protein's interactions with the inhibitor.

First, let's load 2JIE into PyMOL and investigate how the catalytic residues interact with the transition state inhibitor. We will be creating our theozyme using the relative positions of each catalytic residue with the transition state inhibitor.

The catalytic residues in 2JIE are:

* GLU 164 (Acid)
* TYR 295 (Backup)
* GLU 353 (Nucleophile)

In [2]:
# Load params and pose as in 07.03-Ligand-Docking-PyRosetta.ipynb
G2F_params_path = os.path.join('Assets', 'Tutorial_4', 'G2F.params')
complex_pdb_path = os.path.join('Assets', 'Tutorial_4', '2jie-clean.pdb')

def empty_pose_with_params(params_path):
    pose = pyrosetta.rosetta.core.pose.Pose()
    ligand_params = pyrosetta.Vector1([params_path])
    residue_type_set = pose.conformation().modifiable_residue_type_set_for_conf()
    residue_type_set.read_files_for_base_residue_types(ligand_params)
    pose.conformation().reset_residue_type_set_for_conf(residue_type_set)
    return pose
    
complex_pose = empty_pose_with_params(G2F_params_path)
rosetta.core.import_pose.pose_from_file(complex_pose, complex_pdb_path)
pymover.apply(complex_pose)

Use PyMol to measure the geometric relationships between the catalytic residues and the transition state inhibitor G2F bound to PDB ID 2JIE. We are specifically interested in the relationships between:

* Atom OH of TYR 295 and atom OE1 of GLU 353
* Atom OE1 of GLU 353 to atom C1 of G2F

We will consider GLU 164 in a moment. What measurements would you use to describe the geometric relationships between the catalytic residues and G2F? (Hint: Use the measurement modes in PyMOL)

Let's load our new substrate pNPG (it will show up as LG1, ligand 1, in PyMOL) and overlay it onto G2F. Doing this will give us a better idea of how the catalytic residues will interact with the transition state of our desired substrate.

In [3]:
# Overlay pNPG onto G2F
LG1_pdb_path = os.path.join('Assets', 'Tutorial_4', 'LG1_0001.pdb')

pNPG_pose = empty_pose_with_params(LG1_params_path)
rosetta.core.import_pose.pose_from_file(pNPG_pose, LG1_pdb_path)
pymover.apply(pNPG_pose)

Besides the suger pucker, we can overlay the beta-glucoside ring of pNPG onto G2F quite well! Measure the geometric relationships between the catalytic residues and the corresponding atoms in the pNPG substrate we have overlayed onto G2F. In addition to the previous relationships, let's also measure the geometries between atom OE1 in GLU 164 and atom O2 in LG1.

### Defining a Theozyme in a Constraints File

We will be using the geometric relationships between each catalytic residue and pNPG to create a [constraints file](https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/match-cstfile-format). This file defines how atoms in a catalytic residue interact with atoms in the transition state (or another catalytic residue) by defining degrees of freedom (e.g. distances, angles, dihedrals) that must be satisfied.

We can use six degrees of freedom (one distance, two angles, three dihedrals) between any six atoms (three from a catalytic residue, three from the transition state or another catalytic residue) to define a unique geometric configuration. If we define fewer degrees of freedom, there are more possible solutions for arranging sets of atoms that satisfy these constraints.

The atoms and degrees of freedom that were used to define the constraints for E353 and LG1 are visualized here:

<br>
<img src="Assets/Tutorial_4/2JIE-E353-Constraints.png" width="1000" align="center"/>
<br>

Constraints for E164 to LG1 and Y295 to E353 are defined in a similar fashion. The resulting constraints file that defines our theozyme is as follows:

In [4]:
# Print the constraint file
cst_path = os.path.join('Assets', 'Tutorial_4', 'LG1.enzdes.cst')
with open(cst_path, 'r') as cst_file:
    print(cst_file.read())

# Chemical constraints for bagel with 4-nitrophenyl-beta-glucopyranoside
# Alex Carlin, Siegel Lab, UCD
# modified sjb 5-31-15 for matching

# Glu 353 to LG1

CST::BEGIN

  TEMPLATE::    ATOM_MAP:   1     atom_name: C5 O2 C4
  TEMPLATE::    ATOM_MAP:   1     residue3: LG1

  TEMPLATE::    ATOM_MAP:   2     atom_name: OE2 CD CG
  TEMPLATE::    ATOM_MAP:   2     residue1: E 

  CONSTRAINT::  distanceAB:    2.0  0.3  500.0     1  0    
  CONSTRAINT::  angle_A:     180.0 20.0  500.0   360  0 
  CONSTRAINT::  angle_B:     120.0 20.0  500.0   360  0 
  CONSTRAINT::  torsion_B:  -180.0 30.0  500.0   360  0 
  CONSTRAINT::  torsion_AB:  180.0 30.0  500.0   360  0
  CONSTRAINT::  torsion_A:   -42.0 30.0  500.0   360  0

CST::END

# Glu 164 to LG1

CST::BEGIN

  TEMPLATE::    ATOM_MAP:   1     atom_name:  O2 C7 C9
  TEMPLATE::    ATOM_MAP:   1     residue3:   LG1

  TEMPLATE::    ATOM_MAP:   2     atom_name:  OE2 CD CG
  TEMPLATE::    ATOM_MAP:   2     residue1:   E

  CONSTRAINT::  distanceAB: 

The distance/angle/dihedral values in this constraint file have been rounded "in accordance with fundamental organic chemistry principles." How far off are these values from the ones you measured in PyMOL? Why do you think six degrees of freedom are defined for GLU 353 to LG1, but fewer are defined for GLU 164 to LG1 and TYR 295 to GLU 353?

## RosettaMatch

Now that we have defined our theozyme as a constraints file, we need to find a protein that can act as a scaffold to our catalytic residues. [RosettaMatch](https://onlinelibrary.wiley.com/doi/full/10.1110/ps.062353106) is an algorithm implemented in Rosetta that will take a protein backbone and attempt to find a pocket where the transition state can be bound. RosettaMatch simultaneously checks that catalytic residues can be built in this pocket that satisfy the interaction geometries defined in a constraints file.

RosettaMatch iterates through positions in a potential scaffold protein and builds out the transition state from catalytic residue rotamers. Let's consider the constraint for E353 to LG1, but projected into 2D:

<br>
<img src="Assets/Tutorial_4/Matcher-HashSimplification.png" width="800" align="center"/>
<br>

Using the degrees of freedom in the constraint file, we can define the position of the transition state (LG1) relative to a rotamer of glutamate (E353) built at some position in our scaffold protein. For each transition state that we buid out from a rotamer, we will keep track of two things: the position of the transition state and its orientation. In 2D, we will track the position of an atom the transition state (purple circle) as well as a vector (black arrow) that represents the transition state's orientation.

RosettaMatch does not keep track of the exact values for the position and orientation of the transition state; instead, RosettaMatch bins the position and orientation of the transition state. For instance, we can define position bins of size 1 and orientation bins every 5$^\circ$. In the figure above, RosettaMatch would bin the position of the transition state in the (7, 6) position bin and the 90$^\circ$- 95$^\circ$ orientation bin.

As RosettaMatch builds out transition states for catalytic residue rotamers at different positions in our scaffold protein, it will keep track of where the transition state ends up in terms of position and orientation bins. When all catalytic residues in a constraints file build the transition state into the same position and orientation bins, this means that the transition state can be built into the scaffold protein while satisfying the geometric constraints we have defined for each of our catalytic residues. This is considered a match! For example, a successful match for two constraints:

<br>
<img src="Assets/Tutorial_4/Matcher-SuccessfulMatch.png" width="1000" align="center"/>
<br>

Note that the atom on the transition state we have been keeping track of (purple circle) falls into the same position bin for both catalytic residues. Similarly, the orientations of both transition states built from each catalytic residue fall into the same orientation bin. 

A failed match would look like the following:

<br>
<img src="Assets/Tutorial_4/Matcher-FailedMatch.png" width="1000" align="center"/>
<br>

In this case, the transition states for each catalytic residue falls into different position bins and orientation bins. Therefore, it is not possible for the scaffold to accommodate the transition state while satisfying the geometric constraints defined in our constraints file using the current positions and catalytic residue rotamers.

In 3D, RosettaMatch keeps track of where the ligand is built by keeping track of three Euclidian coordinates (x,y,z coordinates of an atom in the transition state) and three [Euler angles](https://en.wikipedia.org/wiki/Euler_angles) (how the transition state is transfromed relative to a reference state). These six values (three Euclidian, three Euler) are binned into discrete values in 6D space, where the position and rotamer that resulted in this configuration are stored. When the transition state is built into the same bin for all constraints in a constraints file, a match is found! This means the theozyme can be built into the scaffold protein at the stored positions with the stored rotamers while still satisfying all defined constraints.

There are several options we can use to tune the outputs of RosettaMatch:

* Eulidian coordinate bin size (-match:euclid_bin_size)
* Euler angle bin size (-match:euler_bin_size)
* Allowed transition state overlap with atoms in the scaffold protein (-bump_tolerance)

Let's use RosettaMatch to build our theozyme into a protein, where 2JIE will act as our scaffold. 

In [11]:
# File Paths and Import Pose
scaffold_path = os.path.join('Assets', 'Tutorial_4', '2jie-scaffold.pdb')
cst_file_path =  str(os.path.join('Assets', 'Tutorial_4', 'LG1.enzdes.cst'))
posfile_path = str(os.path.join('Assets', 'Tutorial_4', 'positions.pos'))
scaffold_pose = rosetta.core.import_pose.pose_from_file(scaffold_path)
match_directory = os.path.join(os.getcwd(), 'Matches')
os.makedirs(match_directory, exist_ok=True)

# Set options for the Matcher
rosetta.basic.options.set_string_option('match:lig_name', 'LG1')
rosetta.basic.options.set_string_option('match:match_grouper', 'SameSequenceGrouper')
rosetta.basic.options.set_string_option('match:output_format', 'PDB')
rosetta.basic.options.set_file_option('match:scaffold_active_site_residues', posfile_path)
rosetta.basic.options.set_file_option('match:geometric_constraint_file', cst_file_path)
rosetta.basic.options.set_boolean_option('match:consolidate_matches', True)
rosetta.basic.options.set_real_option('match:euclid_bin_size', 1)
rosetta.basic.options.set_real_option('match:euler_bin_size', 10)
rosetta.basic.options.set_real_option('match:bump_tolerance', 0.4)
rosetta.basic.options.set_integer_option('match:output_matches_per_group', 1)
# rosetta.basic.options.set_boolean_option('out:file:write_pdb_title_section_records', False)

# Set up MatcherTask
matchertask = rosetta.protocols.match.MatcherTask()

ligpose = rosetta.core.import_pose.pose_from_file(os.path.join('Assets', 'Tutorial_4', 'LG1_0001.pdb'))
ligres = ligpose.conformation().residue_type(1)
atm1, atm2, atm3 = ligres.select_orient_atoms()

oats = rosetta.utility.vector1_core_id_AtomID()
oats.append(rosetta.core.id.AtomID(atm3, 1))
oats.append(rosetta.core.id.AtomID(atm2, 1))
oats.append(rosetta.core.id.AtomID(atm1, 1))

matchertask.set_upstream_pose(scaffold_pose)
matchertask.set_downstream_pose(ligpose, oats)
matchertask.initialize_from_command_line()

# Run the Matcher
matcher = rosetta.protocols.match.Matcher()
matcher.initialize_from_task(matchertask)
processor = rosetta.protocols.match.output.ProcessorFactory.create_processor(matcher, matchertask)

if matcher.find_hits():
    matcher.process_matches(processor)

for file in os.listdir(os.getcwd()):
    if file.startswith('UM_') and file.endswith('.pdb'):
        shutil.move(os.path.join(os.getcwd(), file), match_directory)

Great! There should be a new directory `PyRosettaInteractiveLabs/Matches` that contains the results of RosettaMatch. Each solution PDB is enumerated and lists the identity and position of each catalytic residue in the order defined in our constraint file. Notice that of the 22 solutions that RosettaMatch found, only one solution recapitulates our theozyme at the original scaffold positions. Inspect the other solutions in PyMOL. Do you think these other solutions are just as valid? Why or why not?

Let's move forward with the match solution that places the catalytic residues in their original positions (E353, E164, and Y295). Before we proceed with redesigning the context of the active site, let's see how closely our match solution agrees with the conformations in PDB ID 2JIE. Run the following cell to load the match into PyMOL.

In [12]:
# Find the correct match in Matches
matches = [match for match in os.listdir(match_directory) if 'E353E164Y295' in match]
assert len(matches) == 1
match_solution_path = os.path.join(match_directory, matches[0])

match_pose = rosetta.core.import_pose.pose_from_file(match_solution_path)
pymover.apply(match_pose)

How close is our match? What RosettaMatch and constraint file settings could you change to find a solution that is a closer match to the catalytic residue geometries in PDB ID 2JIE?

## Redesigning Active Site Context
### Design Contraints
[Rosetta's constraint documentation](https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/constraint-file)


In [ ]:
# Redesign the active site context using the packer and minimization

# Add enzdes contraints

# Relax
# Repack
# Minimize
# Repack
# Minimize

## CoupledMoves

So far we have only introduced methods that design on a fixed backbone

Introduce coupledmoves as a method to alter backbone and sidechain DOFs during design

## Filters and Other Quality Metrics
Introduce filters that can quantify different qualities of a design

## Additional References
* https://www.nature.com/articles/s41580-019-0163-x